création des variables globales à partir des variables des fichiers variant caller, et addition de ceux-ci dans la base de données contenant les variables globales venant des fichiers amplicon.

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_qqplot import pplot
import scipy as sp
import sklearn.cluster
import tqdm
from scipy.stats import zscore, kstest, kruskal

In [2]:
path = r"C:\Users\ahmed\MFE project"
df = pd.read_csv(os.path.join(path, [i for i in os.listdir(path) if i.find("DataBase_Norm_tsv")>-1][0]))
df_amp_train = pd.read_csv(os.path.join(path, [i for i in os.listdir(path) if i.find("Database_Global_Amplicon+DT")>-1][0]))

ordr_amp = sorted(list(dict.fromkeys(df_amp_train["ordre"].values)))
ordr_amp = [i for i in ordr_amp if i!=416]
ordr_amp = [i for i in ordr_amp if i!=465]
ordr_amp = [i for i in ordr_amp if i!=466]
ordr_amp = [i for i in ordr_amp if i!=530]
ordr_amp = [i for i in ordr_amp if i!=531]
ordr_amp = [i for i in ordr_amp if i!=599]

df_amp_train = pd.concat([df_amp_train[df_amp_train["ordre"]==i] for i in ordr_amp])
df_tsv_train = pd.concat([df[df["ordre"]==i] for i in ordr_amp])

In [3]:
df_tsv_train.columns

Index(['Unnamed: 0', 'Ref', 'Variant', 'Genotype', 'Frequency', 'Quality',
       'Coverage', 'Allele Cov', 'annee', 'type', 'date', 'num', 'classe',
       'ordre', 'longueur', 'Frequency_norm', 'Quality_norm', 'Coverage_norm',
       'Allele Cov_norm', 'longueur_norm'],
      dtype='object')

In [4]:
# des fonctions pour extraire les variables globales
def Extract_glob_feat(df, feat, longueur):
    nume = []
    nume.append(np.array([1 for i in df[feat].values if i>0.3]).sum()/longueur)
    nume.append(np.array([1 for i in df[feat].values if i>0.5]).sum()/longueur)
    nume.append(np.array([1 for i in df[feat].values if i>0.7]).sum()/longueur)
    nume.append(np.array([1 for i in df[feat].values if i>0.9]).sum()/longueur)
    df_res = pd.DataFrame(data = [[df[feat].mean(), df[feat].median(), nume[0], nume[1], nume[2], nume[3]]] , columns = [feat+"_mean", feat+"_median", feat+"_"+str(0.3), feat+"_"+str(0.5), feat+"_"+str(0.7), feat+"_"+str(0.9)])
    return df_res
    
def extract_all_var(df, variables, longueur):
    list_df = []
    for feat in variables:
        df_t = Extract_glob_feat(df, feat, longueur)
        list_df.append(df_t)
    df_tot = pd.concat(list_df, axis = 1)
    return df_tot

var_percent = ['Frequency_norm', 'Quality_norm', 'Coverage_norm',
       'Allele Cov_norm', 'longueur_norm']

ordr = list(dict.fromkeys(df_tsv_train["ordre"].values))
resultat = []


for i in ordr:
    df_tru = df_tsv_train[df_tsv_train["ordre"]==i]
    longueur = df_tru["longueur"].values[0]
# information des variables en pourcent
    df_var = extract_all_var(df_tru, var_percent, longueur)

# création de la base de données / CSV
    resultat.append(df_var)
df_resul = pd.concat(resultat)
df_resul.index = range(len(df_resul))

In [5]:
df_resul

,Frequency_norm_mean,Frequency_norm_median,Frequency_norm_0.3,Frequency_norm_0.5,Frequency_norm_0.7,Frequency_norm_0.9,Quality_norm_mean,Quality_norm_median,Quality_norm_0.3,Quality_norm_0.5,...,Allele Cov_norm_0.3,Allele Cov_norm_0.5,Allele Cov_norm_0.7,Allele Cov_norm_0.9,longueur_norm_mean,longueur_norm_median,longueur_norm_0.3,longueur_norm_0.5,longueur_norm_0.7,longueur_norm_0.9
0,0.112768,0.0570,0.060976,0.036585,0.024390,0.024390,0.008890,0.000823,0.012195,0.000000,...,0.012195,0.000000,0.000000,0.000000,0.161355,0.161355,0.0,0.0,0.0,0.0
1,0.302000,0.0350,0.440000,0.240000,0.160000,0.160000,0.155718,0.000427,0.200000,0.080000,...,0.360000,0.120000,0.040000,0.040000,0.047809,0.047809,0.0,0.0,0.0,0.0
2,0.162639,0.0350,0.163934,0.131148,0.098361,0.098361,0.068080,0.000342,0.065574,0.065574,...,0.081967,0.065574,0.032787,0.016393,0.119522,0.119522,0.0,0.0,0.0,0.0
3,0.651333,0.5350,0.888889,0.666667,0.444444,0.444444,0.468511,0.358813,0.666667,0.333333,...,0.777778,0.555556,0.333333,0.222222,0.015936,0.015936,0.0,0.0,0.0,0.0
4,0.118640,0.0360,0.101124,0.089888,0.056180,0.044944,0.026327,0.000331,0.033708,0.022472,...,0.033708,0.022472,0.000000,0.000000,0.175299,0.175299,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,0.503833,0.5030,1.000000,0.666667,0.000000,0.000000,0.329192,0.328700,1.000000,0.000000,...,1.000000,0.666667,0.000000,0.000000,0.009960,0.009960,0.0,0.0,0.0,0.0
412,0.507250,0.4560,0.875000,0.500000,0.375000,0.125000,0.240451,0.161359,0.250000,0.125000,...,0.375000,0.250000,0.000000,0.000000,0.013944,0.013944,0.0,0.0,0.0,0.0
413,0.658300,0.5435,0.900000,0.700000,0.400000,0.400000,0.492620,0.340303,0.700000,0.350000,...,0.850000,0.600000,0.350000,0.300000,0.037849,0.037849,0.0,0.0,0.0,0.0
414,0.648278,0.6160,0.833333,0.666667,0.444444,0.388889,0.328664,0.285944,0.444444,0.333333,...,0.611111,0.333333,0.111111,0.055556,0.033865,0.033865,0.0,0.0,0.0,0.0


In [6]:
# for i in range(len(ordr_amp)):
#     if ordr_amp[i]!=df_resul["ordre"].values[i]:
#         print(i, ordr_amp[i], df_resul["ordre"].values[i])

KeyError: 'ordre'

In [7]:
df_amp_train.index = range(len(df_amp_train))
df_global = pd.concat([df_amp_train, df_resul], axis = 1)

In [8]:
df_global.columns

Index(['Unnamed: 0', 'gc_percent_mean', 'gc_percent_median', 'gc_percent_0.3',
       'gc_percent_0.5', 'gc_percent_0.7', 'gc_percent_0.9',
       'cov20_percent_mean', 'cov20_percent_median', 'cov20_percent_0.3',
       'cov20_percent_0.5', 'cov20_percent_0.7', 'cov20_percent_0.9',
       'cov100_percent_mean', 'cov100_percent_median', 'cov100_percent_0.3',
       'cov100_percent_0.5', 'cov100_percent_0.7', 'cov100_percent_0.9',
       'cov500_percent_mean', 'cov500_percent_median', 'cov500_percent_0.3',
       'cov500_percent_0.5', 'cov500_percent_0.7', 'cov500_percent_0.9',
       'fwd_e2e_percent_mean', 'fwd_e2e_percent_median', 'fwd_e2e_percent_0.3',
       'fwd_e2e_percent_0.5', 'fwd_e2e_percent_0.7', 'fwd_e2e_percent_0.9',
       'rev_e2e_percent_mean', 'rev_e2e_percent_median', 'rev_e2e_percent_0.3',
       'rev_e2e_percent_0.5', 'rev_e2e_percent_0.7', 'rev_e2e_percent_0.9',
       'ordre', 'type', 'date', 'classe', 'num2', 'longueur', 'OPT_percent',
       'Frequency_norm_mean

In [9]:
# df_global.to_csv("Database_Global_AmpliconDT_TSV.csv")